In [1]:
import pygame
import pygame_menu

def main_menu(window):
    menu = pygame_menu.Menu('Physics Simulator', 400, 400,
                            theme=pygame_menu.themes.THEME_DARK)
    menu.add.button('Play', start)
    menu.add.button('Quit', pygame_menu.events.EXIT)

    for event in pygame.event.get():
        if event.type == pygame.QUIT:
            return

    menu.mainloop(window)

    # menu.add.button('Level Select', load_peg_file_menu, False)
    # menu.add.button('Create', create_level, [])
    # menu.add.button('Full Screen', full_screen_toggle)

pygame 2.1.2 (SDL 2.0.18, Python 3.8.5)
Hello from the pygame community. https://www.pygame.org/contribute.html
pygame-menu 4.2.2


In [2]:
def start():
    while True:
        window.fill((255, 255, 255))

        lizard_rect = pygame.Rect((width / 2 - 10, height / 2 - 10), (100, 100))
        pygame.draw.rect(window, (128, 128, 128), lizard_rect)

        for event in pygame.event.get():
            if event.type == pygame.QUIT:
                pygame.quit()
                quit()

        pygame.display.update()
        clock.tick(FPS)

In [ ]:
def main():
    # pygame initialization
    print('Starting pygame window...')
    pygame.init()
    window = pygame.display.set_mode((0, 0), pygame.FULLSCREEN)  # Window size 0,0 because starting in full screen
    width, height = window.get_size()
    pygame.display.set_caption("Lizard Game")

    clock = pygame.time.Clock()
    FPS = 60
    width, height = window.get_size()

    print('Starting main menu...')
    main_menu(window)
    
main()

Starting pygame window...
Starting main menu...
